In [ ]:
import kagglehub

path = kagglehub.dataset_download("warcoder/soyabean-seeds")
print("Path to dataset files:", path)

100%|█████████████████████████████████████████████████████████████████████████████| 86.1M/86.1M [00:09<00:00, 9.83MB/s]

Extracting files...


Path to dataset files: C:\Users\GRISH PATIL\.cache\kagglehub\datasets\warcoder\soyabean-seeds\versions\2


In [1]:
import os 

In [3]:
print(os.listdir("soyabean-seeds"))

['.ipynb_checkpoints', '2.complete', 'versions']


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


dataset_path = r'C:\Users\GRISH PATIL\OneDrive\Desktop\GEN AI WITH DS\AgriAihackathon\soyabean-seeds\versions\2\Soybean Seeds'

filepaths = []
labels = []
folds = os.listdir(dataset_path)
for fold in folds:
    f_path = os.path.join(dataset_path, fold)
    if os.path.isdir(f_path):
      files = os.listdir(f_path)
      for file in files:
          filepaths.append(os.path.join(f_path, file))
          labels.append(fold)

df = pd.DataFrame({'filepath': filepaths, 'label': labels})

print(f"Total images found: {len(df)}")
print("Sample of the DataFrame:")
print(df.head())

Total images found: 5513
Sample of the DataFrame:
                                            filepath            label
0  C:\Users\GRISH PATIL\OneDrive\Desktop\GEN AI W...  Broken soybeans
1  C:\Users\GRISH PATIL\OneDrive\Desktop\GEN AI W...  Broken soybeans
2  C:\Users\GRISH PATIL\OneDrive\Desktop\GEN AI W...  Broken soybeans
3  C:\Users\GRISH PATIL\OneDrive\Desktop\GEN AI W...  Broken soybeans
4  C:\Users\GRISH PATIL\OneDrive\Desktop\GEN AI W...  Broken soybeans


In [ ]:
import os

current_path = r'C:\Users\GRISH PATIL\OneDrive\Desktop\GEN AI WITH DS\AgriAihackathon\soyabean-seeds'

print(os.listdir(current_path))

['.ipynb_checkpoints', '2.complete', 'versions']


In [ ]:
train_df, temp_df = train_test_split(df, train_size=0.8, shuffle=True, random_state=42, stratify=df['label'])

valid_df, test_df = train_test_split(temp_df, train_size=0.5, shuffle=True, random_state=42, stratify=temp_df['label'])

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(valid_df)}")
print(f"Testing samples: {len(test_df)}")

train_gen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_gen = ImageDataGenerator(rescale=1./255.)

train_data = train_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

valid_data = val_test_gen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Training samples: 4410
Validation samples: 551
Testing samples: 552
Found 4410 validated image filenames belonging to 5 classes.
Found 551 validated image filenames belonging to 5 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False, 
    weights='imagenet'
)
base_model.trainable = False 

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(train_data.class_indices), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,
    verbose=1
)

model.save('seed_classifier_model.h5')

C:\Users\GRISH PATIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.6331 - loss: 0.9122 - val_accuracy: 0.7586 - val_loss: 0.6399
Epoch 2/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 131s 950ms/step - accuracy: 0.7354 - loss: 0.6709 - val_accuracy: 0.7967 - val_loss: 0.5621
Epoch 3/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 136s 985ms/step - accuracy: 0.7624 - loss: 0.6109 - val_accuracy: 0.7713 - val_loss: 0.5539
Epoch 4/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 127s 922ms/step - accuracy: 0.7844 - loss: 0.5632 - val_accuracy: 0.8040 - val_loss: 0.4996
Epoch 5/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 126s 911ms/step - accuracy: 0.7971 - loss: 0.5325 - val_accuracy: 0.8149 - val_loss: 0.4894
Epoch 6/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 123s 893ms/step - accuracy: 0.8023 - loss: 0.5184 - val_accuracy: 0.8004 - val_loss: 0.4896
Epoch 7/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 131s 949ms/step - accuracy: 0.8093 - loss: 0.4908 - val_accuracy: 0.8167 - val_loss: 0.4760
Epoch 8/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 129s 938ms/step - accuracy: 0.8147 - lo